<a href="https://colab.research.google.com/github/vectara/example-notebooks/blob/main/notebooks/vectara-agentic-intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducing Vectara-Agentic: building AI assistants with Vectara

## About Vectara

[Vectara](https://vectara.com/) is the trusted GenAI RAG-as-a-service platform, providing a simple API for enterprise developers to created RAG applications that are enterprise ready.

See the [Vectara API documentation](https://docs.vectara.com/docs/) for more information on how to use the API.

The main benefits for using Vectara for a RAG application are:
* **Easy to use**: Vectara provides an end-to-end, fully functional, highly scalable and robust RAG pipeline, so as a user you don't have to code up these pieces and maintain them over time.
* **Scalable and Secure**: building GenAI applications may seem easy at first, but the DIY approach can become overwhelming beyond simple examples. Vectara provides instant scalablility to millions of documents, while maintaing data security and privacy, as well as latency SLAs.

## About Vectara-Agentic

Agentic RAG is a method that combines the strengths of RAG with autonomous agents.

With vanilla RAG, Vectara receives a user query, retrieves the most relevant facts from your data, and uses an LLM to generate the most accurate response based on those facts.

Agentic RAG leverages an LLM to “manage” the process of answering the user query via reasoning, planning, and a provided set of “tools”. Since a “manager” LLM-powered agent is in charge, it is smart enough to analyze the user query and properly call tools to obtain a comprehensive response to a complex user query.

[Vectara-agentic](https://vectara.github.io/vectara-agentic-docs/index.html) is a Python package that allows you to build Agentic RAG applications quickly with Vectara. 

## Getting Started

To get started you will need to:
1. [sign up](https://console.vectara.com/signup?utm_source=vectara&utm_medium=signup&utm_term=DevRel&utm_content=example-notebooks&utm_campaign=vectara-signup-DevRel-example-notebooks) (if you haven't already) and follow our [quickstart](https://docs.vectara.com/docs/quickstart) guide to create a corpus and an API key. 

2. Install vectara-agentic using: `pip install vectara-agentic`

Then, setup your Vectara customer ID, corpus ID and API key as environment variables:

In [7]:
vectara_api_key = "<YOUR_VECTARA_API_KEY>"
vectara_corpus_id = "<YOUR_VECTARA_CORPUS_ID>"
vectara_customer_id = "<YOUR_VECTARA_CUSTOMER_ID>"

## The dataset

Vectara is a RAG-as-a-service platform, and in most cases data may be uploaded to the index ahead of time (e.g. by using [Airbyte](https://docs.airbyte.com/integrations/destinations/vectara), [Datavolo](https://vectara.com/blog/building-genai-enterprise-apps-with-vectara-and-datavolo/) or directly via Vectara's [indexing API](https://docs.vectara.com/docs/api-reference/indexing-apis/indexing) or using tools like [vectara-ingest](https://github.com/vectara/vectara-ingest)).

For this notebook we have uploaded [customer complaint data](https://huggingface.co/datasets/hpe-ai/customer-complaints) from HuggingFace into a Vectara corpus, including the following filtering attributes on the corpus to fit the dataset: `Company`, `State` and `Product`.

## Creating our AI Assistant

`vectara-agentic` provide a very flexible interface to create AI assistants and AI agents by defining one or more tools, custom instructions and personalizing the agent to a specific area.

You can read more about this [in the documentation](https://vectara.github.io/vectara-agentic-docs/usage.html).

In this notebook we demonstrate the simplest way to create an AI assistant, utilizing a single RAG Vectara tool, using the `from_corpus()` function:

In [2]:
from vectara_agentic.agent import Agent
from IPython.display import display, HTML

assistant1 = Agent.from_corpus(
    vectara_customer_id=vectara_customer_id,
    vectara_corpus_id=vectara_corpus_id,
    vectara_api_key=vectara_api_key,
    tool_name="ask_complaints",
    data_description="customer complaints",
    assistant_specialty="customer service representative",
    vectara_summarizer="mockingbird-1.0-2024-07-16"
)

Initializing vectara-agentic version 0.1.9...
No observer set.


Note that we've used Vectara's [Mockingbird](https://vectara.com/blog/mockingbird-a-rag-and-structured-output-focused-llm/) LLM by specifying the `vectara_summarizer` argument.

Let's ask the agent a simple question:

In [3]:
res = assistant1.chat("what are the top complaints about credit scores?")
display(HTML(f"<pre>{res}</pre>"))

Now let's create a differnet AI assistant, this time with adding two other arguments to the Vectara RAG Tool:
1. A company name against which the complaint was filed.
2. In which state was the complaint filed

We do this by specifying the vectara_filter_fields. This auto-magically adds these to the Vectara Tool that the agent will use, and the agent will be aware of these fields and use them as it needs to satisfy user queries.

Note that here we also use `verbose=True` to get more details about which steps the AI assistant is taking.

In [4]:
assistant2 = Agent.from_corpus(
    vectara_customer_id=vectara_customer_id,
    vectara_corpus_id=vectara_corpus_id,
    vectara_api_key=vectara_api_key,
    tool_name="ask_complaints",
    data_description="customer complaints",
    assistant_specialty="customer service representative",
    vectara_summarizer="mockingbird-1.0-2024-07-16",
    verbose=True,
    vectara_filter_fields=[
        {'type': 'str', 'name': 'Company', 'description': 'the company related to this complaint'},
        {'type': 'str', 'name': 'State', 'description': 'the state of the complaint, in shorthand format (e.g. TX for Texas)'}
    ]
)

No observer set.


In [5]:
res = assistant2.chat("top complaints about credit scores related to Experian Information Solutions Inc.")
display(HTML(f"<pre>{res}</pre>"))

Added user message to memory: top complaints about credit scores related to Experian Information Solutions Inc.
=== Calling Function ===
Calling function: ask_complaints with args: {"query":"top complaints about credit scores","Company":"Experian Information Solutions Inc."}
Got output: 
                    Response: '''Based on the provided sources, the top complaints about credit scores are:

1. Inaccurate credit scores: Different credit bureaus provide different credit scores, which can be inconsistent and unreliable [1].
2. Failure to report on-time payments: Mortgage companies and creditors may fail to report on-time payments, which can lower credit scores and prevent refinancing [2].
3. Erroneous and unverified information: Credit bureaus may report erroneous and unverified information, which can hurt credit scores [3].
4. Lack of transparency and accountability: Credit bureaus may not provide clear and accurate responses to consumer complaints, and may not remove or update erron

Notice how the AI assistant extracts the company name from the user query, and uses that as an argument to the RAG tool.

`Calling function: vectara_20 with args: {"query":"top complaints about credit scores","Company":"Experian Information Solutions Inc."}`

Similarly let's try with state:

In [6]:
assistant2.chat("top complaints about credit scores exist from California")
display(HTML(f"<pre>{res}</pre>"))

Added user message to memory: top complaints about credit scores exist from California
=== Calling Function ===
Calling function: ask_complaints with args: {"query":"top complaints about credit scores","State":"CA"}
Got output: 
                    Response: '''Based on the provided sources, the top complaints about credit scores are:

1. Inaccurate information on credit reports, including late payments, incorrect account information, and incorrect credit scores [2, 4, 6, 8].
2. Illegal tampering with credit scores, resulting in significant drops in credit scores [3, 5].
3. Lack of transparency and communication from credit bureaus and creditors, including failure to provide verifiable proof of accounts and failure to correct inaccurate information [2, 4, 8].
4. Punitive credit evaluation practices that damage customers' creditworthiness, even with excellent payment histories [10].
5. Difficulty in obtaining credit or obtaining credit at high interest rates due to inaccurate credit inf

Similarly here we have:

`Calling function: vectara_20 with args: {"query":"top complaints about credit scores","State":"CA"}`

So the assistant is automatically identifying california as CA and calling the tool appropriately.